# Lab 3

**ПІДСУМОВУВАННЯ ТЕКСТУ ІЗ ЗАСТОСУВАННЯМ МОДЕЛЕЙ HUGGING FACE**

In [42]:
from datasets import load_dataset
import evaluate
import pandas as pd
from transformers import pipeline
from transformers import (PegasusTokenizer, PegasusForConditionalGeneration,
                          T5Tokenizer, T5ForConditionalGeneration)

In case you need to login

In [2]:
from huggingface_hub import login

login()

Load ROUTE metric

In [3]:
rouge = evaluate.load("rouge")

## Create Summarizer

In [5]:
model_name = "google/pegasus-cnn_dailymail"

In [6]:
model_name = "facebook/bart-large-cnn"

In [63]:
model_name = "Falconsai/text_summarization"

In [5]:
model_name = "madushakv/t5_xsum_samsum_billsum_cnn_dailymail"

In [64]:
summarizer = pipeline("summarization", model=model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


## Summarize

In [66]:
with open("backlash_as_new_eu_political_ad_rules_kick_in.txt", 'r') as f:
    text = f.read()

In [61]:
with open("rating_26_years_of_java_changes.txt", 'r') as f:
    text = f.read()

In [67]:
summarizer(text)

[{'summary_text': "Google, Meta and Microsoft have opted to stop showing political ads in the EU altogether . Politicians on both sides of the aisle said it could be detrimental to democratic debate . At the heart of the EU's attempt is a bid to curb political manipulation and foreign interference during elections ."}]

## Use Directly

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [68]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Encode input text for the model

In [69]:
inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
inputs

{'input_ids': tensor([[    3,  6934,  3063, 23143,   134,     3,   318,  8767,  1611,  3545,
          2219,  3855,    12,  1172, 13567,   300,   367, 20356,    43,     3,
             7, 16669,     3,     9,  6772,    13, 12334,     6,    38,   779,
          5357,  6979,   323,  1827,  6543,  1446,    13,  9095,    53,     5,
         18409,   277,   497,     8,   973,    56,  1137,     3,     9, 10947,
          1453,    13,   251,   227,    34,     3, 18728,   688,   379,  1163,
             6, 14204,    11,  2803,    12,  4028,     3,     9,  1001,   670,
            30,  1827,  3662,     5, 14984,  7137,    30,   321,  4458,    13,
             8, 23597,   243,    34,   228,    36, 28879,    12, 15053,  5054,
             5,    37,  3527,   243,    34,    19,  2718,    13,     8,  2261,
          3315,    11,    19,  6168,  6927,    28,  2734,  7130,   688,    12,
         21002,     8,    73,    23, 25916, 11737,     5,   486,     8,   842,
            13,     8,  3371,    31,  

Generate summary with custom parameters

In [70]:
summary_ids = model.generate(
    **inputs,
    max_length=30,  # maximum length of the summary
    min_length=10,  # minimum length of the summary
    num_beams=4,  # beam search for better results
    length_penalty=1.0,  # penalize very long summaries
    early_stopping=True,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

'to improve transparency around online ads have sparked a wave of criticism, as major platforms shut down political ads instead of complying'

## Experiment

In [71]:
summary_ids = model.generate(
    **inputs,
    min_length=3,
    max_length=5,
    length_penalty=0.1,
)
summary_ids

tensor([[    0, 32099,     6,  1163,   243]])

Limited

In [72]:
summary_ids = model.generate(
    **inputs,
    min_length=3,
    max_length=5,
    length_penalty=0.1,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

', Google said'

Talkative

In [35]:
summary_ids = model.generate(
    **inputs,
    min_length=1000,
    max_length=1000,
    length_penalty=0.1,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

'Fresh European Union rules intended to improve transparency around online advertisements have sparked a wave of criticism. Campaigners say the law will cause a harmful loss of information after it triggered companies to blackout. Politicians on both sides of the aisle said it could be detrimental to democratic debate. Google, Meta and Microsoft have all opted to stop showing political ads in the EU altogether. Politicians on both sides of the aisle said it could be detrimental to democratic debate. The Commission said it is aware of the serious concerns and is continuing talks with Big Tech companies to mitigate the unintended impacts. The law aims to improve transparency around online advertisements have sparked a wave of criticism. Google, Meta and Microsoft have opted to stop showing political ads in the EU altogether. The law is an example of over-regulation gone wild. The law sparked a wave of criticism. The law is intended to improve transparency around online advertisements. Th

Penalized

In [73]:
summary_ids = model.generate(
    **inputs,
    min_length=100,
    max_length=200,
    length_penalty=5.0,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

', Google said the definition of political advertising is too broad, while Meta criticized targeted ad restrictions. Politicians on both sides of the aisle said it could be detrimental to democratic debate. The Commission said it is aware of the serious concerns and is continuing talks with Big Tech companies to mitigate the unintended impacts. The new law on Transparency and Targeting of Political Advertising, which kicked in on Friday, brings new restrictions and transparency requirements for paid political ads.'

Stable

In [74]:
summary_ids = model.generate(
    **inputs,
    do_sample=False,
    max_length=200,
    min_length=50,
    length_penalty=1.0,  # Neutral
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=3,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

'to improve transparency around online ads have sparked a wave of criticism, as major platforms shut down political ads instead of complying. Campaigners say the law will cause a harmful loss of information after it triggered companies including Google, Meta and Microsoft to implement a blackout on political advertising. Politicians on both sides of the aisle said it could be detrimental to democratic debate.'

Slightly creative

In [75]:
summary_ids = model.generate(
    **inputs,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,  # Nucleus sampling
    max_length=200,
    min_length=50,
    length_penalty=1.2,  # Slight preference for completeness
    no_repeat_ngram_size=3,
)
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

"Law, which kicked in on Friday, brings new restrictions and transparency requirements for paid political ads. Campaigners say the law will cause a harmful loss of information. At the heart of the EU's attempt is a bid to curb political manipulation and foreign interference during elections."

## Load Dataset

Take first 5 examples for demo

In [39]:
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0", split="train[:5]")
dataset

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 5
})

In [76]:
data = []
for i, sample in enumerate(dataset):
    text = sample["article"]
    reference_summary = sample["highlights"]
    generated_summary = summarizer(
        text,
        min_length=30,
        max_new_tokens=100,
    )[0]["summary_text"]
    data.append({
        "example_id": i + 1,
        "article": text,
        "reference_summary": reference_summary,
        "generated_summary": generated_summary
    })

In [77]:
df = pd.DataFrame(data)
df

,example_id,article,reference_summary,generated_summary
0,1,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Daniel Radcliffe says he has no plans to fritt...
1,2,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Inmates with the most severe mental illnesses ...
2,3,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","survivor: ""I probably had a 30-, 35-foot free ..."
3,4,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...","""none appeared worrisome,"" White House spokesm..."
4,5,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...","NFL commissioner: ""Your admitted conduct was n..."


## Evaluate

Compute ROUGE scores using the results

In [79]:
results = rouge.compute(
    predictions=df["generated_summary"],
    references=df["reference_summary"],
)
results

{'rouge1': np.float64(0.3059582620927158),
 'rouge2': np.float64(0.1269496855345912),
 'rougeL': np.float64(0.22430467136349486),
 'rougeLsum': np.float64(0.25920752139239533)}